In [210]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

- 分析10天上涨超过8的机会, 在何时出现
    - sz_jt_10： 0,1,2,3： 0, 0-3，3-8, 8-...  
- sql statement
    - "SELECT * from day_zhibiao where 
    - jx_days_ud60 > 20 and jx_days_ud60 < 40 and \
    - jx_xl_60>-"+str(xl_dgree)+" and jx_xl_10>0 and \
    - lj_fl_1_3 > 1.1 and lj_fl_3_10>1.05 "
- 暂定4个主要特征：
    - fl_1_3, fl_3_10, pct_dg_5, pct_dg_10

In [211]:
sql = "SELECT * \
from ds_qf60"
print(sql)
df = pd.read_sql_query(sql, conn, index_col=None)

df.shape

SELECT * from ds_qf60


(40730, 47)

In [212]:
# df.info()

In [213]:
col_list_ds = ['symbol', 'trade_date', 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
    'sz_max_5','sz_max_10','sz_max_20',
    'sz_jt_10','class0' ]
df.tail(1)[col_list_ds]

,symbol,trade_date,pct_change,diff,dea,bar,jx_days_ud60,jx_dg_120,jx_dg_60,jx_dg_20,...,jx_zs_60,pct_fl_1_3,pct_fl_3_10,pct_fl_5_20,pct_fl_20_60,sz_max_5,sz_max_10,sz_max_20,sz_jt_10,class0
40729,689009,2021-03-31,2.24,-3.69456,-5.00514,2.62117,27,0.0,-0.130428,-0.329783,...,0,1.10161,1.05168,0.885532,0.434366,10.1262,10.1262,10.1262,3.0,3


In [214]:
df['sz_jt_10'].value_counts(), df['sz_jt_10'].value_counts()/len(df)

(2.0    12597
 1.0    11721
 3.0     8622
 0.0     7790
 Name: sz_jt_10, dtype: int64,
 2.0    0.309281
 1.0    0.287773
 3.0    0.211687
 0.0    0.191260
 Name: sz_jt_10, dtype: float64)

In [215]:
# 开发测试：条件取数
# num = len(df)

# cond2a = df['sz_jt_10']>=2
# df[ cond2a ]['sz_jt_10'].value_counts()

# cond2b = df['sz_jt_10']>1 
# cond2c = df['sz_jt_10']<=2
# cond2bc2 = ( cond2b & cond2c )
# df[ cond2bc2 ]['sz_jt_10'].value_counts()

In [216]:
# epoch 初始化数据
# col = ['pct_change',
#     'diff', 'dea',\
#     'jx_days_ud60',\
#     'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
#     'jx_zs_5', 'jx_zs_10', 'jx_zs_20', \
#     'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
#     'sz_max_10' ]
col = ['jx_dg_5','pct_fl_1_3','step', 'class', 'rate' ]

def print_header():
    print('- jx_days_ud60 > 20 and jx_days_ud60 < 40 ')
    print('- jx_xl_60>-"+str(xl_dgree)+" and jx_xl_10>0 ')
    print('- pct_fl_1_3 > 1.1579 and pct_fl_3_10>1.0730 ')
    print('-'*50)
    print('  2周   1周  :%14s %14s %14s %14s'%('10均线角度   ','3-10日放量   ','5日均线角度   ','1-3日放量   '),\
        ' = %5s,%6s,%5s'%('数量','达成','成功%'))
    print(' ----  ----  :',' '*2, '-'*10,' '*6, '-'*10,' '*6, '-'*10,' '*6, '-'*10,' '*2)

In [262]:
# 设置循环步长等
steps = 5
steps_dg_10, steps_fl_3_10 = 1,1
steps_dg_5, steps_fl_1_3   = 1,1

# jx_dg_10 预处理
min_dg_10, max_dg_10 = df['jx_dg_10'].min(), df['jx_dg_10'].max()
step_dg_10 = (max_dg_10 - min_dg_10)/steps_dg_10
# fl_3_10 预处理
min_fl_3_10, max_fl_3_10 = df['pct_fl_3_10'].min(), df['pct_fl_3_10'].max()
step_fl_3_10 = (max_fl_3_10 - min_fl_3_10)/steps_fl_3_10

# jx_dg_5预处理
min_dg_5, max_dg_5 = df['jx_dg_5'].min(), df['jx_dg_5'].max()
step_dg_5 = (max_dg_5 - min_dg_5)/steps_dg_5
# fl_1_3 预处理
min_fl_1_3, max_fl_1_3 = df['pct_fl_1_3'].min(), df['pct_fl_1_3'].max()
step_fl_1_3 = (max_fl_1_3 - min_fl_1_3)/steps_fl_1_3

In [272]:
%%time
# man[col] = 0.0
man_data = {'jx_dg_5':0.0,
            'pct_fl_1_3':0.0,
            'step':0, 'class':0, 'rate':0.0    
}
man = pd.DataFrame(data=man_data, index=[0])
print_header()
all_step, all_class = 0, 0
# 开始epoch

cond_jt_10_3 = df['sz_jt_10']>=3
# cond_last = cond_begin

from_dg_10 = min_dg_10
for i_dg_10 in range(steps_dg_10):
    to_dg_10 = min_dg_10 + (i_dg_10+1)*step_dg_10
    # 生成条件
    cond_dg_10 = (df['jx_dg_10'] >= from_dg_10) & (df['jx_dg_10'] < to_dg_10 )
    # cond_last = (cond_dg_10) & ( cond_last )
    cond_last = (cond_dg_10) 

    from_dg_5 = min_dg_5
    for i_dg_5 in range(steps_dg_5):
        to_dg_5 = min_dg_5 + (i_dg_5+1)*step_dg_5
        # 生成条件
        cond_dg_5 = (df['jx_dg_5'] >= from_dg_5) & (df['jx_dg_5'] < to_dg_5 )
        cond_last = (cond_dg_5) & ( cond_last )
        
        from_fl_3_10 = min_fl_3_10
        for i_fl_3_10 in range(steps_fl_3_10):
            to_fl_3_10 = min_fl_3_10 + (i_fl_3_10+1)*step_fl_3_10
            # 生成条件
            cond_fl_3_10 = (df['pct_fl_3_10'] >= from_fl_3_10) & (df['pct_fl_3_10'] < to_fl_3_10 )
            cond_last = (cond_fl_3_10) & ( cond_last )
        
            from_fl_1_3 = min_fl_1_3
            for i_fl_1_3 in range(steps_fl_1_3):
                to_fl_1_3 = min_fl_1_3 + (i_fl_1_3+1)*step_fl_1_3
                # 生成条件
                cond_fl_1_3 = (df['pct_fl_1_3'] >= from_fl_1_3) & (df['pct_fl_1_3'] < to_fl_1_3 )
                cond_last = ( cond_fl_1_3 ) & (cond_last)
                
                # 核心运算               
                cond_purpose = (cond_jt_10_3) & ( cond_last )
                num_step_series  = df[ cond_last ]['sz_jt_10'].value_counts() 
                num_class_series = df[ cond_purpose ]['sz_jt_10'].value_counts() 
                num_step = num_step_series.iloc[0] if not num_step_series.empty else 0
                num_class = num_class_series.iloc[0] if not num_class_series.empty else 0
                num_class_rate = 0 if num_class==0 else num_class*100/num_step
                print( '%2d %2d %2d %2d '%(i_dg_10, i_fl_3_10, i_dg_5, i_fl_1_3), \
                    ':%8.4f,%8.4f,%8.4f,%8.4f,%8.4f,%8.4f,%8.4f,%8.4f'%(from_dg_10, to_dg_10, from_fl_3_10, to_fl_3_10, \
                        from_dg_5, to_dg_5, from_fl_1_3, to_fl_1_3),\
                    ' = %7.0f,%7.0f,%7.2f'%(num_step, num_class, num_class_rate) )
                # 更新到man
                all_step  += num_step  ; all_class += num_class
                man_list = [from_dg_5, from_fl_1_3, num_step, num_class, num_class_rate]
                man = pd.DataFrame(np.insert(man.values, len(man.index), values=man_list, axis=0))        
                # 下一个 fl_1_3 loop
                from_fl_1_3 += step_fl_1_3
            # 下一个 fl_3_10 loop
            from_fl_3_10 += step_fl_3_10            
        # jx_dg_5下一个loop
        from_dg_5 += step_dg_5
    # jx_dg_10下一个loop
    from_dg_10 += step_dg_10
    
all_class_rate = all_class*100/all_step 
print( '-'*112)
print( ' '*86, '= %7.0f, %7.0f, %5.2f'%(all_step, all_class, all_class_rate) )
man.shape


- jx_days_ud60 > 20 and jx_days_ud60 < 40 
- jx_xl_60>-"+str(xl_dgree)+" and jx_xl_10>0 
- pct_fl_1_3 > 1.1579 and pct_fl_3_10>1.0730 
--------------------------------------------------
  2周   1周  :     10均线角度        3-10日放量         5日均线角度         1-3日放量     =    数量,    达成,  成功%
 ----  ----  :    ----------        ----------        ----------        ----------   
 0  0  0  0  :  0.0000,  2.7200,  1.0500, 34.4487, -1.8567,  6.2110,  1.1000, 66.3237  =   12597,   8620,  68.43
----------------------------------------------------------------------------------------------------------------
                                                                                       =   12597,    8620, 68.43
CPU times: total: 0 ns
Wall time: 24.3 ms


(2, 5)

In [265]:
""" 运算结果
  dg_10: 0-2.72， 上面可能可以更高   df['jx_dg_10']>=0.5440) & (df['jx_dg_10']<2.4480)

""" 
df_total = df[ ( df['jx_dg_10']>=0) & (df['jx_dg_10']<2.72) \
    ]['sz_jt_10'].value_counts()
df_total, df_total.sum(), df_total[3], df_total[3]/df_total.sum()*100

(2.0    12597
 1.0    11721
 3.0     8622
 0.0     7789
 Name: sz_jt_10, dtype: int64,
 40729,
 8622,
 21.169191485182548)

In [266]:
# 数据核对 - 1
df[ (df['sz_jt_10']>=3) ]['sz_jt_10'].value_counts()

3.0    8622
Name: sz_jt_10, dtype: int64

In [267]:
df[ (df['jx_dg_10']>=0) & (df['jx_dg_10']<0.4533) & (df['sz_jt_10']>=3) ]['sz_jt_10'].value_counts()


3.0    6938
Name: sz_jt_10, dtype: int64

In [268]:
df[ (df['jx_dg_10']>=0.4533) & (df['jx_dg_10']<2.7200) & (df['sz_jt_10']>=3) ]['sz_jt_10'].value_counts()

3.0    1684
Name: sz_jt_10, dtype: int64

In [269]:
df[ (df['jx_dg_10']>=0.4533) & (df['jx_dg_10']<2.7200) &\
    (df['jx_dg_5']>=-1.8567) & (df['jx_dg_5']<6.2110) &
    (df['sz_jt_10']>=3) ]['sz_jt_10'].value_counts()


3.0    1684
Name: sz_jt_10, dtype: int64

In [270]:
pd.set_option('display.max_rows', None)
man.columns = col
# man.sort_index(ascending=True)
# man.set_index('rate', drop=False)
sorting = man.sort_values('rate', ascending=False)
sorting

,jx_dg_5,pct_fl_1_3,step,class,rate
1,-1.85673,1.10001,12597.0,8620.0,68.428991
0,0.00000,0.00000,0.0,0.0,0.000000


In [271]:
cond_fl_1_3_class.value_counts()

NameError: name 'cond_fl_1_3_class' is not defined

In [ ]:
# man